In [2]:
import numpy as np
import pandas as pd
import re
import datetime
import os
data_path='/Users/rony/Projects/Health_Geo_app/data/raw_data/QoF/'
file_names=[fn for fn in os.listdir(data_path) if (('xlsx' in fn) & ('1819' not in fn))]
file_names=[fn for fn in os.listdir(data_path) if 'xlsx' in fn]
file_line_drop={'qof-1314-prev-all-levels.xlsx': [11,12],'qof-1617-prev-all-lev.xlsx':[9,10], 
'qof-1415-prev-all-levels.xlsx':[11,12], 'qof-1516-prev-all-lev-v2.xlsx':[7,8],\
'qof-1718-prev-all-lev.xlsx':[9,10],\
'qof-1819-prev-all-lev.xlsx': [12,13,14,15]}

practices_columns_drop={'qof-1314-prev-all-levels.xlsx':13,'qof-1617-prev-all-lev.xlsx':11, 
'qof-1415-prev-all-levels.xlsx':13, 'qof-1516-prev-all-lev-v2.xlsx':9,\
'qof-1718-prev-all-lev.xlsx':11,\
'qof-1819-prev-all-lev.xlsx':16} 

In [3]:
def get_indications (xl):
    sheet_names=xl.sheet_names
    #print('sheet names:',sheet_names)
    new_headers=['Register','Prevalence']
    menu_indications=[]
    main_headers=['Practice code']+new_headers+['Indication']
    indications_df=pd.DataFrame(columns=main_headers)
    
    for index, sheet_name in enumerate (sheet_names[1:]): 
        if 'Prac' in sheet_name:
            #print ('sheet_name:',sheet_name)
            df=xl.parse(sheet_name)#,skiprows=8, failed for 1516
            
            #search indications assuming the form DEM - Dementia
            headers=list(df.columns)
            
            # Get the row with indication headers
            for index, row in df.iterrows():
                row=row.values.tolist()
                row=[str(item) for item in row if str(item)!='nan']
                indications=[item for item in row if re.findall('[A-Z]\s-\s[A-Z][a-z]',item)]
                if indications:
                    #print ('indications in row:',indications)
                    indications_index=index
                    indications_header=df.iloc[indications_index:indications_index+1,:].values.tolist()[0]
                    break

            # Get practices code columns
            for header in df.columns:
                items=list(df[header])
                
                #if 'Practice code' in items:
                ch=[h for h in items if re.findall('Practice [C|c]ode',str(h))]
                if ch: 
                    rows_to_skip = practices_columns_drop[file_name]
                    practices_column=items
                    #print ('rows_to_skip:',rows_to_skip)
                    #print ('practices_column before skip:',practices_column[:15])
                    practices_column=practices_column[rows_to_skip:]
                    #print ('practices_column:',practices_column[:15])
                    break
                
            df=df.iloc[indications_index+1:,:]
            indications_header=[re.sub('\(\)','',str(h)) for h in indications_header]
            df.columns=list(indications_header)
            for indication in indications:
                #print ('indication collected:',indication)
                from_column=df.columns.get_loc(indication)
                to_column=df.columns.get_loc(indication)+4
                indication_df=df.iloc[:,from_column:to_column]
                indication_df=indication_df.iloc[:,2:]
                indication_df.columns=new_headers
                #print ('df before header lines drop')
                #print (indication_df.head(9))
                #print (indication_df.info())
                lines_to_drop=file_line_drop[file_name]
                #print ('lines_to_drop',lines_to_drop)
                indication_df=indication_df.drop(lines_to_drop) 
                #print ('df after header lines drop')
                #print ('number of df columns:',len(indication_df.columns))
                #print ('indication headers:', list(indication_df.columns))
                #print (indication_df.head())
                #print (indication_df.info())
                
                #print (len (indication_df), len (practices_column))
                indication_df['Practice code']=practices_column
                indication_df['Indication'] = indication
                indication_df=indication_df[main_headers]
                
                #print ('\n\ndata with practices codes')
                #print (indication_df.head())
                #print (30*'*')
                indications_df=pd.concat([indications_df,indication_df],sort=False)
                #print ('All indications=',set(indications_df['Indication']))
                #print (30*'*')
    return indications_df

In [8]:
results_path='/Users/rony/Projects/Health_Geo_app/data/QoF/'
indications_all_years = []
for file_name in file_names:
    restuls_file_name='{rp}prevalence_20{ys}.csv'\
    .format (rp=results_path,ys=re.findall('\d{2}(\d{2})',file_name)[0]) # Ex: 1314->2014
    print (restuls_file_name)
    data_file='{dp}{fn}'.format(dp=data_path,fn=file_name)
    xl = pd.ExcelFile(data_file)
    indications_df=get_indications (xl)
    new_main_headers=['Practice code','Indication','Register','Prevalence']
    indications_df=indications_df[new_main_headers]
    numeric_columns=indications_df[['Register','Prevalence']]
    # #print identify none numeric values
    #print ('\n*** exploring none numeric values ***\n')
    
    for header in numeric_columns.columns:
        #print (header)
        not_numbers=[i for i in numeric_columns[header] if \
                     not re.findall ('\d{1,}',str(i))]
        #print ("not numbers count:",len (not_numbers))
        #print (pd.Series(not_numbers).value_counts())
    #print ('df pre-clean')
    #print (indications_df.info())
    to_nan=[':','-','']
    for item in to_nan:
        indications_df.replace(item, np.nan, inplace=True)
    
    indications_df.dropna(inplace=True)#(subset=['Register'])
    #indications_df=indications_df[~indications_df.isin([':','-',''])]
    indications_df['Prevalence']=indications_df['Prevalence'].astype(float,errors = 'ignore')
    indications_df['Prevalence'] = round (indications_df['Prevalence'],3)
    indications_df['Register']=indications_df['Register'].astype(float,errors = 'ignore')
    indications_df['Register'] = round (indications_df['Register'],3)
    
    indications_df['Indication']=[re.sub(' - ','_',i) for i in indications_df['Indication']]
    indications_df['Indication']=[re.sub('\s','_',i) for i in indications_df['Indication']]
    indications_df['Indication']=[re.sub(',','',i) for i in indications_df['Indication']]
    
    #print ('df post-clean')
    indications = list (set (indications_df['Indication']))
    print ('indications:',indications)
    print (indications_df.info())
    print (indications_df.head(10))

    print (30*'+'+'\n'+30*'+')
    
    indications_all_years += indications    
    
    indications_df.to_csv(restuls_file_name,index=False) 

indications_all_years = list (set (indications_all_years))
print ('indications_all_years:',indications_all_years)

/Users/rony/Projects/Health_Geo_app/data/QoF/prevalence_2019.csv
indications: ['EP_Epilepsy_(18+)', 'DM_Diabetes_Mellitus_(17+)', 'COPD_Chronic_Obstructive_Pulmonary_Disease', 'STIA_Stroke_and_Transient_Ischaemic_Attack', 'OB_Obesity_(18+)', 'CAN_Cancer', 'CHD_Coronary_Heart_Disease', 'CVD-PP_Cardiovascular_Disease_Primary_Prevention_(30-74)', 'DEM_Dementia', 'HF_Heart_Failure', 'MH_Mental_Health', 'RA_Rheumatoid_Arthritis_(16+)', 'AST_Asthma', 'OST_Osteoporosis_(50+)', 'PAD_Peripheral_Arterial_Disease', 'HYP_Hypertension', 'LVSD_Heart_Failure_due_to_Left_Ventricular_Systolic_Dysfunction', 'DEP_Depression_(18+)', 'LD_Learning_Disabilities', 'AF_Atrial_Fibrillation', 'PC_Palliative_Care', 'CKD_Chronic_Kidney_Disease_(18+)']
<class 'pandas.core.frame.DataFrame'>
Int64Index: 151206 entries, 16 to 6888
Data columns (total 4 columns):
Practice code    151206 non-null object
Indication       151206 non-null object
Register         151206 non-null float64
Prevalence       151206 non-null floa

/Users/rony/Projects/Health_Geo_app/data/QoF/prevalence_2016.csv
indications: ['LVSD_Heart_Failure_due_to_Left_Ventrical_Systolic_Dysfunction', 'EP_Epilepsy_(18+)', 'DM_Diabetes_Mellitus_(17+)', 'COPD_Chronic_Obstructive_Pulmonary_Disease', 'STIA_Stroke_and_Transient_Ischaemic_Attack', 'CAN_Cancer', 'CHD_Coronary_Heart_Disease', 'CVD-PP_Cardiovascular_Disease_Primary_Prevention_(30-74)', 'DEM_Dementia', 'HF_Heart_Failure', 'MH_Mental_Health', 'RA_Rheumatoid_Arthritis_(16+)', 'AST_Asthma', 'OST_Osteoporosis_(50+)', 'PAD_Peripheral_Arterial_Disease', 'OB_Obesity', 'HYP_Hypertension', 'DEP_Depression_(18+)', 'LD_Learning_Disabilities', 'AF_Atrial_Fibrillation', 'PC_Palliative_Care', 'CKD_Chronic_Kidney_Disease_(18+)']
<class 'pandas.core.frame.DataFrame'>
Int64Index: 167591 entries, 9 to 7627
Data columns (total 4 columns):
Practice code    167591 non-null object
Indication       167591 non-null object
Register         167591 non-null float64
Prevalence       167591 non-null float64
dtype

In [9]:
with open ('deprivation_indices.txt','w') as f:
    for indication in indications_all_years:
        f.write ('{i}\n'.format (i=indication))